In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow
from tensorflow import keras
from keras.layers import Flatten, Dense, Input, Dropout, Add
from keras.preprocessing.image import ImageDataGenerator
from keras import Model
from keras.regularizers import l2

In [ ]:
datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2, zoom_range=[0.8, 1.2],
                             horizontal_flip=True,vertical_flip=True, validation_split=0.2)

In [ ]:
path = '/content/drive/MyDrive/Extras/New Terrain Images'

In [ ]:
train_data = datagen.flow_from_directory(path, subset='training', target_size=(229,229), shuffle=True)
val_data = datagen.flow_from_directory(path, subset='validation', target_size=(229,229), shuffle=True )

Found 439 images belonging to 4 classes.
Found 107 images belonging to 4 classes.


In [ ]:
from keras.applications.xception import Xception
input = Input(shape=(229,229,3))
model = Xception(include_top=False, weights='imagenet', input_tensor=input)
model.trainable=False

In [ ]:
output = model.output
x = Flatten()(output)
x = Dense(units = 256, activation='relu', kernel_regularizer=l2(0.01))(x)
x = Dropout(0.3)(x)
x = Dense(units = 125, activation='relu', kernel_regularizer=l2(0.01))(x)
x = Dropout(0.3)(x)
x = Dense(units = 4, activation='softmax')(x)
final_model = Model(inputs=input, outputs=x)

In [ ]:
final_model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])

In [ ]:
final_model.fit(train_data, batch_size=32, epochs=200, validation_data=val_data)

Epoch 1/200
14/14 [==============================] - 14s 776ms/step - loss: 7.9602 - accuracy: 0.4282 - val_loss: 7.4029 - val_accuracy: 0.5421
Epoch 2/200
14/14 [==============================] - 9s 627ms/step - loss: 7.3708 - accuracy: 0.5421 - val_loss: 6.8835 - val_accuracy: 0.6262
Epoch 3/200
14/14 [==============================] - 10s 706ms/step - loss: 6.9189 - accuracy: 0.5877 - val_loss: 6.5205 - val_accuracy: 0.6636
Epoch 4/200
14/14 [==============================] - 10s 726ms/step - loss: 6.4813 - accuracy: 0.6128 - val_loss: 6.1756 - val_accuracy: 0.6729
Epoch 5/200
14/14 [==============================] - 9s 676ms/step - loss: 6.0834 - accuracy: 0.6538 - val_loss: 5.8412 - val_accuracy: 0.6542
Epoch 6/200
14/14 [==============================] - 9s 659ms/step - loss: 5.8541 - accuracy: 0.6515 - val_loss: 5.7004 - val_accuracy: 0.6822
Epoch 7/200
14/14 [==============================] - 10s 723ms/step - loss: 5.6371 - accuracy: 0.6492 - val_loss: 5.3174 - val_accuracy: 0.

In [ ]:
final_model.save('/content/drive/MyDrive/Extras/Terrain.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
final_model.save('/content/drive/MyDrive/Extras/Terrain_model.keras')

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/Extras/Terrain.h5')

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import image
import numpy as np

In [ ]:
# Load the new image
sandy_img = '/content/drive/MyDrive/Extras/sandy terrain.jpg'
marshy_img_path = '/content/drive/MyDrive/Extras/grass terrain.jpg'
rocky_img = '/content/drive/MyDrive/Extras/rocky terrain.jpg'
grass_img = '/content/drive/MyDrive/Extras/grass terrain.jpg'
img = image.load_img(grass_img, target_size=(229, 229))  # Adjust target_size as needed

# Convert the image to a numpy array
img_array = image.img_to_array(img)

# Expand the dimensions to match the input shape of your model
img_array = np.expand_dims(img_array, axis=0)

# Perform any additional preprocessing required by your model (e.g., normalization)
img_array = img_array / 255.0  # Normalize pixel values if necessary


In [ ]:
predictions = model.predict(img_array)

1/1 [==============================] - 0s 51ms/step


In [ ]:

# Decode the predictions (assuming it's a classification model)
class_labels = ['Grass Terrain', 'Marshy', 'Rocky', 'sandy']  # Replace with your actual class labels
predicted_class_index = np.argmax(predictions)
predicted_class_label = class_labels[predicted_class_index]

In [ ]:
predicted_class_index

2

In [ ]:
# Retrieve the class indices
class_indices = train_data.class_indices

# Print the class indices
print("Class Indices:")
print(class_indices)


Class Indices:
{'Grass Terrain': 0, 'Marshy Terrain': 1, 'Rocky Terrain': 2, 'Sandy Terrain': 3}


In [ ]:
!pip install gradio

In [ ]:
sandy_img = '/content/drive/MyDrive/Extras/sandy terrain.jpg'
marshy_img = '/content/drive/MyDrive/Extras/Marshy Terrain.jpg'
rocky_img = '/content/drive/MyDrive/Extras/rocky terrain.jpg'
grass_img = '/content/drive/MyDrive/Extras/grass terrain.jpg'

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import image
import numpy as np
import gradio as gr
import numpy as np
import cv2

model = keras.models.load_model('/content/drive/MyDrive/Extras/Terrain.h5')

def recognize(input_img):

    img = tf.image.resize(input_img, (229, 229))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0

    predictions = model.predict(img_array)

    class_labels = ['Grass Terrain', 'Marshy Terrain', 'Rocky Terrain', 'Sandy Terrain']
    predicted_class_index = np.argmax(predictions)
    predicted_class_label = class_labels[predicted_class_index]

    img = convert_to_grayscale(input_img)
    # img = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)
    # # img = cv2.imread(input_img, cv2.IMREAD_GRAYSCALE)

    gx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3)
    gy = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3)
    grad = np.sqrt(gx**2 + gy**2)

    roughness = np.std(grad)

    # # # print('Surface roughness:', roughness)

    return img_array[0], predicted_class_label, roughness

examples = [sandy_img, rocky_img, grass_img, marshy_img]
demo = gr.Interface(fn=recognize, inputs='image', outputs=['image', 'text', 'text'], examples=examples)

demo.launch(debug=True, share=False)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Keyboard interruption in main thread... closing server.


In [ ]:
import numpy as np
import cv2

# Load the image
img = cv2.imread(marshy_img, cv2.IMREAD_GRAYSCALE)

# Calculate the gradient
gx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3)
gy = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3)
grad = np.sqrt(gx**2 + gy**2)

# Calculate the standard deviation of the gradient
roughness = np.std(grad)

print('Surface roughness:', type(roughness))

Surface roughness: <class 'numpy.float64'>


In [ ]:
from keras.applications import InceptionV3
input = Input(shape=(224,224,3))
model = InceptionV3(include_top=False, weights='imagenet', input_tensor=input)
model.trainable=False

87910968/87910968 [==============================] - 5s 0us/step


In [ ]:
output = model.output
x = Flatten()(output)
x = Dense(units = 256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(units = 125, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(units = 4, activation='softmax')(x)
final_model = Model(inputs=input, outputs=x)

In [ ]:
final_model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])

In [ ]:
final_model.fit(train_data, batch_size=32, epochs=200, validation_data=val_data)

Epoch 1/200
8/8 [==============================] - 188s 21s/step - loss: 1.5575 - accuracy: 0.4585 - val_loss: 1.3437 - val_accuracy: 0.5818
Epoch 2/200
8/8 [==============================] - 1s 137ms/step - loss: 0.9897 - accuracy: 0.6725 - val_loss: 0.8778 - val_accuracy: 0.6727
Epoch 3/200
8/8 [==============================] - 1s 142ms/step - loss: 0.5779 - accuracy: 0.7904 - val_loss: 0.8788 - val_accuracy: 0.6909
Epoch 4/200
8/8 [==============================] - 1s 137ms/step - loss: 0.3963 - accuracy: 0.8384 - val_loss: 1.0466 - val_accuracy: 0.6727
Epoch 5/200
8/8 [==============================] - 1s 136ms/step - loss: 0.2451 - accuracy: 0.9170 - val_loss: 0.9117 - val_accuracy: 0.6545
Epoch 6/200
8/8 [==============================] - 1s 142ms/step - loss: 0.2447 - accuracy: 0.9214 - val_loss: 0.9626 - val_accuracy: 0.7273
Epoch 7/200
8/8 [==============================] - 1s 143ms/step - loss: 0.1747 - accuracy: 0.9214 - val_loss: 0.9210 - val_accuracy: 0.7091
Epoch 8/200
8